In [1]:
import pandas as pd
import numpy as np
import json
import random

In [2]:
DIR = './dataset/train/'

texts = {
    'id': [],
    'text-id': [],
    'text': [],
    'num-authors': [],
    'level': [],
    'change' : []
}


In [3]:
for level in range(1,4,1):
    for i in range(1,901, 1):
        filename = f'problem-{i}.txt'
        with open('dataset\pan23-multi-author-analysis-dataset'+ str(level) + '\pan23-multi-author-analysis-dataset'+ str(level) + '-validation\\' + filename, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            lines = " ||| ".join(lines)

        json_filename = f'truth-problem-{i}.json'
        with open('dataset\pan23-multi-author-analysis-dataset'+ str(level) +'\pan23-multi-author-analysis-dataset'+ str(level) + '-validation\\' + json_filename, 'r', encoding='utf-8') as json_file:
            json_data = json.load(json_file)
            num_authors = json_data['authors']  # Get the maximum number of authors
            change_array = json_data['changes']

        if level == 1:
            id = 'easy-' + str(i)
        elif level == 2:
            id = 'medium-'+str(i)
        elif level == 3:
            id = 'hard-'+str(i)
        
        texts['id'].append(id)
        texts['text-id'].append(i)
        texts['text'].append(lines)
        texts['num-authors'].append(num_authors)
        texts['level'].append(level)
        texts['change'].append(change_array)

In [4]:
data = pd.DataFrame(texts)
data['text-length'] = data['text'].apply(lambda x: len(x))
data

,id,text-id,text,num-authors,level,change,text-length
0,easy-1,1,I think the premise of your question is off to...,2,1,[1],1551
1,easy-2,2,"Also, I'm hoping that enough of the usual Fox ...",4,1,"[1, 1, 1, 1]",1047
2,easy-3,3,"In the 1970s, the US tried to intervene severa...",2,1,"[1, 1]",1062
3,easy-4,4,"As far as I am aware, there is no consensus ov...",2,1,"[0, 0, 1]",1772
4,easy-5,5,I think alot of this is natural. This is calle...,2,1,[1],1274
...,...,...,...,...,...,...,...
2695,hard-896,896,There's actually many different ways a tank ca...,2,3,"[0, 1, 0, 0]",1745
2696,hard-897,897,"ok, and, as the article mentions, the vast maj...",4,3,"[1, 1, 0, 1, 0, 0, 0, 0, 0]",1500
2697,hard-898,898,"Ok, I went through the online thingy to self-v...",2,3,"[0, 0, 0, 1]",1196
2698,hard-899,899,They're not stupid. Without US support and pro...,2,3,[1],1095


In [5]:
data.describe()

,text-id,num-authors,level,text-length
count,2700.000000,2700.000000,2700.000000,2700.000000
mean,450.500000,3.000000,2.000000,2038.946296
std,259.855587,0.816648,0.816648,1747.848674
min,1.000000,2.000000,1.000000,1007.000000
25%,225.750000,2.000000,1.000000,1170.750000
50%,450.500000,3.000000,2.000000,1418.500000
75%,675.250000,4.000000,3.000000,2024.500000
max,900.000000,4.000000,3.000000,10180.000000


In [6]:
isolated_text = data.loc[(data['text-length'] >= data['text-length'].quantile(0.25)) & (data['text-length'] <= data['text-length'].quantile(0.75)) & (data['level'] >= 2)]
isolated_text.shape

(898, 7)

In [7]:
random.seed(39)
check_up_text = random.choice(isolated_text.loc[(isolated_text['num-authors'] == 3) & (isolated_text['level'] == 3), 'id'].tolist())
check_up_text

'hard-252'

In [8]:
chosen_id = []
random.seed(39)
for i in range(32):
    texts_id = random.choice(isolated_text['id'].tolist())
    chosen_id.append(texts_id)

In [9]:
group_size = 4
for i in range(0, len(chosen_id), group_size):
    print("Version", (i // group_size) + 1)
    for j in range(group_size):
        if i + j < len(chosen_id):
            print("    {}".format(chosen_id[i + j]))
    print('-'*60)

Version 1
    medium-533
    medium-677
    hard-51
    medium-84
------------------------------------------------------------
Version 2
    medium-492
    medium-564
    hard-72
    hard-656
------------------------------------------------------------
Version 3
    hard-370
    medium-17
    hard-501
    hard-784
------------------------------------------------------------
Version 4
    medium-659
    hard-555
    hard-3
    hard-624
------------------------------------------------------------
Version 5
    medium-445
    hard-808
    medium-85
    medium-746
------------------------------------------------------------
Version 6
    hard-547
    hard-862
    medium-192
    hard-24
------------------------------------------------------------
Version 7
    medium-31
    medium-269
    hard-121
    medium-889
------------------------------------------------------------
Version 8
    hard-118
    hard-762
    hard-693
    medium-253
--------------------------------------------------------

In [10]:
# The longest texts
data.loc[data['text-length'] == data.loc[data['id'].isin(chosen_id), 'text-length'].max()]

,id,text-id,text,num-authors,level,change,text-length
984,medium-85,85,"""I would like to have this conversation over p...",4,2,"[1, 1, 0, 0, 1, 1, 1, 1, 1]",2024


In [11]:
file_path = 'dataset_validation_allLevels.csv'
# Save the DataFrame as CSV
data.to_csv(file_path, index=False)